In [19]:
# as MatchMS can't parse the Passatutto files, I created my own parser which matches spectra to spectrum objects from MatchMS
# it is available under my FDR-Metabolomics repo: src/passatutto_parser.py
import sys
sys.path.append(r'C:\Users\Gosia\Desktop\FDR-Metabolomics\src\passatutto_parser.py')
import passatutto_parser as pp
# Taking all the files from MassBankOrbi Passatutto (queries) and parsing them to json objects
pre_spectrums = pp.PassatuttoParser(r'C:\Users\Gosia\Downloads\MassbankOrbi').parse_folder()

processed 100 files
processed 200 files
processed 300 files
processed 400 files
Finished parsing of 457 spectra 


In [20]:
# my parser creates json objects, so the next step is to convert them to spectra
from matchms.importing.load_from_json import as_spectrum
spectrums = []
for i, s in enumerate( pre_spectrums ):
    spectrums.append(as_spectrum(s))
    if i and i % 100 == 0:
        print('processed %d'%i)

processed 100
processed 200
processed 300
processed 400


In [21]:
# creating a dict with compound name and it's inchi
from pprint import pprint
query_name_inchi = {}
for s in spectrums:
    query_name_inchi[s.get("compound_name")] = s.get("inchi")

In [22]:
from rdkit.Chem.inchi import InchiToInchiKey,MolToInchiKey
#these two files are provided by Passatutto people. They contain: target, query, target inchi and q-value
#random_q_vals = open(r'C:\Users\Gosia\Desktop\q_values\MassbankOrbi-Gnps_qValues_EBA.txt', 'r').readlines()  
random_q_vals = open(r'C:\Users\Gosia\Desktop\q_values\MassbankOrbi-Gnps_qValues_TDA_RandomPeaks.txt', 'r').readlines() 

# I tried several ways of comparing inchis, these two seem to be the closest to original but not good enough
def inchisEqual(i1, i2):
    return i1.split("/")[:4] == i2.split("/")[:4]
    #return InchiToInchiKey(i1).split('-')[0] == InchiToInchiKey(i2).split('-')[0]

# for easiness of access to appropriate parts of the hit I created a named tuple (object)
from collections import namedtuple
Hit = namedtuple('Hit', ['query', 'score', 'false_hit', 'inchi'])
    
hits = []
name_q_val = {}
for line in random_q_vals[1:]:
    q,t,ti,s,q_val = line.split("\t") # q for query, t for target, ti for target inchi, s for score
    false_hit = not inchisEqual( query_name_inchi[q], ti) # IMPORTANT: this is where the inchi comparison happens, for viewing you can uncomment print below
    #print("inchi query:\n %s \n inchi target:\n %s \n rdkit inchi target:\n %s \n rdkit inchi query:\n %s" % (query_name_inchi[q],ti,InchiToInchiKey(name_inchi[q]),InchiToInchiKey(ti)))
    name_q_val.setdefault(q, []).append( (q_val,ti) )
    hits.append(Hit(q,s,false_hit,ti))

In [23]:
# this cell just shows we have a few compounds with the same name and different q-values
from collections import Counter
c = Counter( h.query for h in hits )
pprint( c.most_common(3) )

[('Erythromycin', 2), ('Reserpine', 2), ('Adenosine', 2)]


In [24]:
# Q-value calculation
QVal = namedtuple( 'QVal', [ 'query', 'qval', 'inchi', 'false_hit'])

def calculate_q_value(hits):
    hits.sort(key=lambda x: x.score, reverse=True)
    fdr_vals = []
    for i in range(len(hits)):
        #calculate the proportion of false hits so far
        fdr = sum([int(h.false_hit) for h in hits[:i+1]])/(i+1)
        fdr_vals.append(fdr)
    q_vals = [0 for f in fdr_vals]
    q_vals[-1] = fdr_vals[-1]
    # ensure non-decreasing q_value
    for i in range(len(q_vals)-2,0,-1):
        q_vals[i] = min(fdr_vals[i],q_vals[i+1])
    q_list = []
    for i,h in enumerate(hits):
        # Simon says: if not h.false_hit:
        q_list.append(QVal(h.query, q_vals[i], h.inchi, h.false_hit))
    return fdr_vals, q_vals, q_list

In [25]:
# calculation of q-values
import numpy as np
_,_,q_values = calculate_q_value(hits)
#for qval in q_values:
#    print(qval.qval, qval.false_hit)

In [26]:
# this function handles repeating compounds with different q-values and takes the smaller q-value for representation
def take_smaller(x):
    # name_q_val needs to be repopulated after running this cell
    if len(x) == 1:
        return float(x[0][0])
    smaller = 0
    for i, (n,_) in enumerate(x):
        if float(n) < float(x[smaller][0]):
            smaller = i
    return float(x.pop(smaller)[0])

count = 0
print('Passatutto,   Ours,      Are They the same?     Name')
for name,q_val,inchi,_ in q_values:
    print('%4f      %4f    %s            %s' %( take_smaller(name_q_val[name]), float(q_val), inchisEqual(name_inchi[name], inchi),name))
    

Passatutto,   Ours,      Are They the same?     Name
0.000000      0.000000    True            Bupropion
0.000000      0.000000    True            Ketoprofen
0.000000      0.000000    True            Sotalol
0.000000      0.000000    True            Flufenamic acid
0.000000      0.000000    True            Moclobemide
0.000000      0.000000    True            Mefenamic acid
0.000000      0.000000    True            Gabapentin
0.000000      0.000000    True            Venlafaxine
0.000000      0.000000    True            Clarithromycin
0.000000      0.000000    True            Metoclopramide
0.000000      0.000000    True            Fluconazole
0.000000      0.000000    True            Terbutryn
0.000000      0.000000    True            Clozapine
0.000000      0.000000    True            1-(3-(Trifluoromethyl)phenyl)piperazine
0.000000      0.000000    True            Sulfathiazole
0.000000      0.000000    True            Amisulpride
0.000000      0.000000    True            Sulfapyrid

Thoughts:
1. How is the jump from q-value 0.264469 to 0.334322 possible from Passatutto data? (L-Phenylalanine --> Acetamiprid, RandomPeaks file)
2. Final q-values are not very different
3. They seem to have a different way of identifying True/False hits and/or a completely different q-value calculation. Their codebase doesn't seem to provide a clear explanation of how either is done (although q-value calculation seems similar to ours).

# Aminopyrine example:
This is the first compound for them that seems to be a false hit. Inchi strings are exactly the same though.

Query:  Aminopyrine InChI=1S/C13H17N3O/c1-10-12(14(2)3)13(17)16(15(10)4)11-8-6-5-7-9-11/h5-9H,1-4H3

Target: Amidopyrine InChI=1S/C13H17N3O/c1-10-12(14(2)3)13(17)16(15(10)4)11-8-6-5-7-9-11/h5-9H,1-4H3



# Sebuthylazine example
This is a compound that doesn't change their q-value and inch strings as well as the chemical formula are very different    

Query: Sebuthylazine 

Target: MLS001048955-01!2,2-dimethyl-4,10-dihydro-3H-pyrano[2,3-b]quinolin-5-one

InChI=1S/C9H16ClN5/c1-4-6(3)12-9-14-7(10)13-8(15-9)11-5-2/h6H,4-5H2,1-3H3,(H2,11,12,13,14,15)

InChI=1S/C14H15NO2/c1-14(2)8-7-10-12(16)9-5-3-4-6-11(9)15-13(10)17-14/h3-6H,7-8H2,1-2H3,(H,15,16)